In [621]:
import pandas as pd
import numpy as np

In [622]:
YearlyUPT_df = pd.read_excel('January 2025 Complete Monthly Ridership (with adjustments and estimates)_250303.xlsx', sheet_name=8)
YearlyVRM_df = pd.read_excel('January 2025 Complete Monthly Ridership (with adjustments and estimates)_250303.xlsx', sheet_name=9)
MasterInfo_df = pd.read_excel('January 2025 Complete Monthly Ridership (with adjustments and estimates)_250303.xlsx', sheet_name=1)
AllTransit_df = pd.read_csv('alltransit10k.csv', encoding='utf-8')

In [623]:
# Filter the YearlyUPT_df to only show 'Rail' modes of transportation
YearlyUPT_df_edit = YearlyUPT_df[YearlyUPT_df['3 Mode'] == 'Rail']

# Drop unnecessary columns
YearlyUPT_df_edit = YearlyUPT_df_edit.drop(columns=[
    'Legacy NTD ID',
    'Reporter Type',
    'UACE CD',
    '3 Mode'
    ])
YearlyUPT_df_edit.head(3)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
3,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,357935.0,403590.0,398109.0,374327.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,NaN,NaN,NaN,NaN,...,1358297.0,1417507.0,1685668.0,1863409.0,749443.0,829953.0,1117605.0,1411817.0,1492563.0,122247.0
29,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [624]:
# Transform the dataframe from wide to long data
YearlyUPT_df_edit_long = pd.melt(
    YearlyUPT_df_edit,
    id_vars=['NTD ID', 'Agency','Mode/Type of Service Status', 'UZA Name', 'Mode', 'TOS'],
    var_name='Year',
    value_name='UPT'
)
YearlyUPT_df_edit_long.head(5)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN
3,8.0,Tri-County Metropolitan Transportation Distric...,Active,"Portland, OR--WA",LR,DO,2002,25665600.0
4,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",SR,DO,2002,NaN


In [625]:
# Filter the YearlyVRM_df to only show 'Rail' modes of transportation
YearlyVRM_df_edit = YearlyVRM_df[YearlyVRM_df['3 Mode'] == 'Rail']

# Drop unnecessary columns 
YearlyVRM_df_edit = YearlyVRM_df_edit.drop(columns=[
    'Legacy NTD ID',
    'Reporter Type',
    'UACE CD',
    '3 Mode'
    ])
YearlyVRM_df_edit.head(3)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
3,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,39630.0,43005.0,44994.0,38893.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,NaN,NaN,NaN,NaN,...,191472.0,193810.0,207850.0,205470.0,147792.0,187454.0,180369.0,177754.0,173924.0,16085.0
29,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [626]:
# Transform the dataframe from wide to long data
YearlyVRM_df_edit_long = pd.melt(
    YearlyVRM_df_edit,
    id_vars=['NTD ID', 'Agency','Mode/Type of Service Status', 'UZA Name', 'Mode', 'TOS'],
    var_name='Year',
    value_name='VRM'
)
YearlyVRM_df_edit_long.head(5)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,VRM
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,39630.0
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN
3,8.0,Tri-County Metropolitan Transportation Distric...,Active,"Portland, OR--WA",LR,DO,2002,3198863.0
4,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",SR,DO,2002,NaN


In [627]:
# Merge the YearlyUPT_df_edit_long & YearlyVRM_df_edit_long dataframes using an inner join
merged_UPTVRM_df = pd.merge(
    YearlyUPT_df_edit_long,
    YearlyVRM_df_edit_long,
    how='inner',
    on=['NTD ID', 'Agency','Mode/Type of Service Status', 'UZA Name', 'Mode', 'TOS', 'Year']
)
merged_UPTVRM_df.head(3)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0,39630.0
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN,NaN
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN,NaN


In [628]:
merged_UPTVRM_df.shape

(3456, 9)

In [629]:
# Filter the MasterInfo_df to only show 'Rail' modes of transportation
MasterInfo_df_edit = MasterInfo_df[MasterInfo_df['3 Mode'] == 'Rail']

# Drop unnecessary columns 
MasterInfo_df_edit = MasterInfo_df_edit.drop(columns=[
    'Legacy NTD ID',
    'Mode/Type of Service Status',
    'Reporter Type',
    'UACE CD',
    '3 Mode'
    ])
MasterInfo_df_edit.head(3)

,NTD ID,Agency,Mode,TOS,Organization Type,HQ City,HQ State,UZA Name,UZA SQ Miles,UZA Population,...,Last Closed Report Year,Last Closed FY End Month,Last Closed FY End Year,Passenger Miles FY,Unlinked Passenger Trips FY,Avg Trip Length FY,Fares FY,Operating Expenses FY,Avg Cost Per Trip FY,Avg Fares Per Trip FY
3,1,King County,LR,DO,"City, County or Local Government Unit or Depar...",SEATTLE,WA,"Seattle--Tacoma, WA",982.52,3544011.0,...,2011.0,12.0,2011.0,471587.0,520933.0,0.905274,288470.0,2281393.0,4.379437,0.553756
6,1,King County,SR,DO,"City, County or Local Government Unit or Depar...",SEATTLE,WA,"Seattle--Tacoma, WA",982.52,3544011.0,...,2023.0,12.0,2023.0,1556379.0,1411817.0,1.102394,617795.0,13964269.0,9.890991,0.437589
29,8,Tri-County Metropolitan Transportation Distric...,CR,PT,Independent Public Agency or Authority of Tran...,Portland,OR,"Portland, OR--WA",519.30,2104238.0,...,2011.0,6.0,2011.0,2558105.0,306228.0,8.353596,311656.0,6245998.0,20.396561,1.017725


In [630]:
# Merge the merged_UPTVRM_df & MasterInfo_df_edit dataframes using an inner join
full_merge_df = pd.merge(
    merged_UPTVRM_df,
    MasterInfo_df_edit,
    how='inner',
    on=['NTD ID', 'Agency', 'UZA Name', 'Mode', 'TOS']
)
display(full_merge_df.head(3))
print(full_merge_df.shape)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Last Closed Report Year,Last Closed FY End Month,Last Closed FY End Year,Passenger Miles FY,Unlinked Passenger Trips FY,Avg Trip Length FY,Fares FY,Operating Expenses FY,Avg Cost Per Trip FY,Avg Fares Per Trip FY
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0,39630.0,"City, County or Local Government Unit or Depar...",...,2011.0,12.0,2011.0,471587.0,520933.0,0.905274,288470.0,2281393.0,4.379437,0.553756
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN,NaN,"City, County or Local Government Unit or Depar...",...,2023.0,12.0,2023.0,1556379.0,1411817.0,1.102394,617795.0,13964269.0,9.890991,0.437589
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,2011.0,6.0,2011.0,2558105.0,306228.0,8.353596,311656.0,6245998.0,20.396561,1.017725


(3456, 26)


## Data Cleaning

In [632]:
# Create a copy of the AllTransit_df DataFrame to clean
AllTransit_Clean = AllTransit_df.copy()
AllTransit_Clean.head(3)

,rank,name,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
0,1,"New York, NY",9.6,8.7,1296548,13960,19,44,0.544,8621501
1,2,"Hoboken, NJ",9.5,5.9,1937781,5232,7,37,0.574,58754
2,3,"San Francisco, CA",9.3,7.4,596446,9616,20,35,0.312,851036


In [633]:
# Split the 'name' column into 2 columns, 'city' & 'state'
AllTransit_Clean[['city', 'state']] = AllTransit_Clean['name'].str.split(', ', expand=True)

# Drop the 'name' column
AllTransit_Clean = AllTransit_Clean.drop(columns={'name'})

# View new DataFrame
AllTransit_Clean.head(3)

,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population,city,state
0,1,9.6,8.7,1296548,13960,19,44,0.544,8621501,New York,NY
1,2,9.5,5.9,1937781,5232,7,37,0.574,58754,Hoboken,NJ
2,3,9.3,7.4,596446,9616,20,35,0.312,851036,San Francisco,CA


In [634]:
AllTransit_Cities = AllTransit_Clean['city'].unique()
AllTransit_Cities

array(['New York', 'Hoboken', 'San Francisco', ..., 'Waxhaw', 'Waycross',
       'Dardenne Prairie'], dtype=object)

In [635]:
# Create a copy of the full_merge_df DataFrame to clean
full_merge_clean = full_merge_df.copy()
full_merge_clean.head(3)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Last Closed Report Year,Last Closed FY End Month,Last Closed FY End Year,Passenger Miles FY,Unlinked Passenger Trips FY,Avg Trip Length FY,Fares FY,Operating Expenses FY,Avg Cost Per Trip FY,Avg Fares Per Trip FY
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0,39630.0,"City, County or Local Government Unit or Depar...",...,2011.0,12.0,2011.0,471587.0,520933.0,0.905274,288470.0,2281393.0,4.379437,0.553756
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN,NaN,"City, County or Local Government Unit or Depar...",...,2023.0,12.0,2023.0,1556379.0,1411817.0,1.102394,617795.0,13964269.0,9.890991,0.437589
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,2011.0,6.0,2011.0,2558105.0,306228.0,8.353596,311656.0,6245998.0,20.396561,1.017725


In [636]:
# View cities and states in full_merge_df
print(full_merge_clean["HQ City"].unique(), '\n')
print(full_merge_clean["HQ State"].unique())

['SEATTLE' 'Portland' 'ANCHORAGE' 'PORTLAND' 'BOSTON' 'NEWINGTON'
 'BUFFALO' 'BROOKLYN' 'CAMDEN' 'New York' 'NEWARK' 'NEW YORK'
 'Staten Island' 'JOHNSTOWN' 'PHILADELPHIA' 'PITTSBURGH' 'WASHINGTON'
 'BALTIMORE' 'HARRISBURG' 'WOODBRIDGE' 'HAMPTON' 'MORGANTOWN'
 'CHATTANOOGA' 'MEMPHIS' 'CHARLOTTE' 'ATLANTA' 'MIAMI' 'JACKSONVILLE'
 'TAMPA' 'POMPANO BEACH' 'SAN JUAN' 'MADISON' 'Atlanta' 'TALLAHASSEE'
 'KENOSHA' 'CINCINNATI' 'CLEVELAND' 'MINNEAPOLIS' 'CHICAGO' 'CHESTERTON'
 'DETROIT' 'Detroit' 'MILWAUKEE' 'EL PASO' 'Fort Worth' 'HOUSTON'
 'GALVESTON' 'OKLAHOMA CITY' 'NEW ORLEANS' 'NORTH LITTLE ROCK' 'AUSTIN'
 'DALLAS' 'LEWISVILLE' 'ALBUQUERQUE' 'SAINT LOUIS' 'KANSAS CITY'
 'SALT LAKE CITY' 'DENVER' 'HONOLULU' 'OAKLAND' 'SAN JOSE' 'SAN FRANCISCO'
 'SACRAMENTO' 'SAN DIEGO' 'OCEANSIDE' 'TUCSON' 'San Diego' 'SAN CARLOS'
 'LOS ANGELES' 'STOCKTON' 'PHOENIX' 'Las Vegas' 'PETALUMA'] 

['WA' 'OR' 'AK' 'MA' 'CT' 'ME' 'NY' 'NJ' 'PA' 'DC' 'MD' 'VA' 'WV' 'TN'
 'NC' 'GA' 'FL' 'PR' 'WI' 'OH' 'MN' 'IL' 'IN

In [637]:
# Change all values in 'HQ City' column to title case
full_merge_clean["HQ City"] = full_merge_clean["HQ City"].str.title()

# Review values in 'HQ City' column
full_merge_cities = full_merge_clean["HQ City"].unique()
print(full_merge_cities)

['Seattle' 'Portland' 'Anchorage' 'Boston' 'Newington' 'Buffalo'
 'Brooklyn' 'Camden' 'New York' 'Newark' 'Staten Island' 'Johnstown'
 'Philadelphia' 'Pittsburgh' 'Washington' 'Baltimore' 'Harrisburg'
 'Woodbridge' 'Hampton' 'Morgantown' 'Chattanooga' 'Memphis' 'Charlotte'
 'Atlanta' 'Miami' 'Jacksonville' 'Tampa' 'Pompano Beach' 'San Juan'
 'Madison' 'Tallahassee' 'Kenosha' 'Cincinnati' 'Cleveland' 'Minneapolis'
 'Chicago' 'Chesterton' 'Detroit' 'Milwaukee' 'El Paso' 'Fort Worth'
 'Houston' 'Galveston' 'Oklahoma City' 'New Orleans' 'North Little Rock'
 'Austin' 'Dallas' 'Lewisville' 'Albuquerque' 'Saint Louis' 'Kansas City'
 'Salt Lake City' 'Denver' 'Honolulu' 'Oakland' 'San Jose' 'San Francisco'
 'Sacramento' 'San Diego' 'Oceanside' 'Tucson' 'San Carlos' 'Los Angeles'
 'Stockton' 'Phoenix' 'Las Vegas' 'Petaluma']


In [638]:
# Rename the "HQ City" and "HQ State" columns to "city" and "state"
full_merge_clean = full_merge_clean.rename(columns={
    "HQ City": "city",
    "HQ State": "state"
})
full_merge_clean.columns

Index(['NTD ID', 'Agency', 'Mode/Type of Service Status', 'UZA Name', 'Mode',
       'TOS', 'Year', 'UPT', 'VRM', 'Organization Type', 'city', 'state',
       'UZA SQ Miles', 'UZA Population', 'Service Area Population',
       'Service Area SQ Miles', 'Last Closed Report Year',
       'Last Closed FY End Month', 'Last Closed FY End Year',
       'Passenger Miles FY', 'Unlinked Passenger Trips FY',
       'Avg Trip Length FY', 'Fares FY', 'Operating Expenses FY',
       'Avg Cost Per Trip FY', 'Avg Fares Per Trip FY'],
      dtype='object')

In [639]:
# Convert the cities list for both dataframes into sets
AllTransit_set = set(AllTransit_Cities)
full_merge_set = set(full_merge_cities)

# Find the missing cities using the differnce function
missing_cities = full_merge_set.difference(AllTransit_set)
missing_cities

{'Honolulu', 'Saint Louis', 'Staten Island'}

In [640]:
full_merge_missing = full_merge_clean[full_merge_clean['city'].isin(missing_cities)]
full_merge_missing = full_merge_missing[['city', 'state', 'UZA Population', 'UZA Name', 'Agency']].drop_duplicates().reset_index()
display(full_merge_missing)

,index,city,state,UZA Population,UZA Name,Agency
0,32,Staten Island,NY,19426449.0,"New York--Jersey City--Newark, NY--NJ",Staten Island Rapid Transit Operating Authority
1,106,Saint Louis,MO,2156323.0,"St. Louis, MO--IL",Bi-State Development Agency of the Missouri-Il...
2,107,Saint Louis,MO,2156323.0,"St. Louis, MO--IL",Loop Trolley Transportation Development District
3,114,Honolulu,HI,853252.0,"Honolulu, HI",City and County of Honolulu


In [641]:
AllTransit_Clean.columns

Index(['rank', 'score', 'tci', 'jobs', 'trips/week', 'routes',
       'transit shed (mi2)', '%transit', 'population', 'city', 'state'],
      dtype='object')

In [642]:
# Fix the missing city names in the AllTransit_Clean dataframe 
AllTransit_Clean['city'] = AllTransit_Clean['city'].replace('St. Louis', 'Saint Louis')
AllTransit_Clean['city'] = AllTransit_Clean['city'].replace('Urban Honolulu', 'Honolulu')
AllTransit_Clean['city'] = AllTransit_Clean['city'].replace('East Honolulu', 'Honolulu')

# Replace 'Staten Island' in the full_merge_clean df with 'New York'
full_merge_clean['city'] = full_merge_clean['city'].replace('Staten Island', 'New York')

In [643]:
# Check that there are no longer any missing cities
AllTransit_set = set(AllTransit_Clean['city'].unique())
full_merge_set = set(full_merge_clean['city'].unique())
missing_cities = full_merge_set.difference(AllTransit_set)
print(missing_cities)

set()


In [644]:
# Merge the 2 dataframes
initial_merge = pd.merge(full_merge_clean, AllTransit_Clean, on=['city', 'state'], how='inner')
display(initial_merge.head(3))
initial_merge.shape

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Avg Fares Per Trip FY,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0,39630.0,"City, County or Local Government Unit or Depar...",...,0.553756,95,8.3,4.9,403896,4783,19,38,0.214,734731
1,1.0,King County,Active,"Seattle--Tacoma, WA",SR,DO,2002,NaN,NaN,"City, County or Local Government Unit or Depar...",...,0.437589,95,8.3,4.9,403896,4783,19,38,0.214,734731
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,1.017725,127,8.1,2.6,218492,3517,10,44,0.109,641885


(3408, 35)

In [645]:
# View the number of unique cities in the 'City Column'
initial_merge['city'].nunique()

64

In [646]:
# Drop all rows with a year of 2025 because that's the current year and values only reflect what ridership has been before Feburary 1st 2025
index_to_drop = initial_merge[initial_merge['Year'] == 2025].index
initial_merge = initial_merge.drop(index_to_drop)
initial_merge.shape

(3266, 35)

### Deal with missing values

In [648]:
initial_merge.isna().sum()

NTD ID                            0
Agency                            0
Mode/Type of Service Status       0
UZA Name                          0
Mode                              0
TOS                               0
Year                              0
UPT                            1243
VRM                            1245
Organization Type                 0
city                              0
state                             0
UZA SQ Miles                      0
UZA Population                    0
Service Area Population          23
Service Area SQ Miles            23
Last Closed Report Year           0
Last Closed FY End Month          0
Last Closed FY End Year           0
Passenger Miles FY               69
Unlinked Passenger Trips FY      69
Avg Trip Length FY               69
Fares FY                         69
Operating Expenses FY            46
Avg Cost Per Trip FY             69
Avg Fares Per Trip FY            69
rank                              0
score                       

In [649]:
na_index = initial_merge[initial_merge['UPT'].isna() & initial_merge['VRM'].isna()].index

initial_merge[['Agency','Mode/Type of Service Status','Mode', 'Year','UPT','VRM']].loc[na_index]

,Agency,Mode/Type of Service Status,Mode,Year,UPT,VRM
1,King County,Active,SR,2002,NaN,NaN
2,Tri-County Metropolitan Transportation Distric...,Inactive,CR,2002,NaN,NaN
4,Tri-County Metropolitan Transportation Distric...,Inactive,SR,2002,NaN,NaN
5,Tri-County Metropolitan Transportation Distric...,Active,YR,2002,NaN,NaN
8,Central Puget Sound Regional Transit Authority,Active,LR,2002,NaN,NaN
...,...,...,...,...,...,...
3253,"San Diego Trolley, Inc.",Inactive,LR,2024,NaN,NaN
3255,Southern California Regional Rail Authority,Inactive,CR,2024,NaN,NaN
3259,Altamont Corridor Express,Inactive,CR,2024,NaN,NaN
3263,Las Vegas Monorail Company,Active,MG,2024,NaN,NaN


It appears that some of the NA values are caused by the service not being active or not having been built yet. For example, the Streetcar service in King County opened in 2007 and didn't start reporting until 2012, so the years before 2012 are null. While other NA values are caused by an agency not reporting. We'll want to remove the NA values caused by the service not existing, and impute the null values where an agency doesn't report.

In [651]:
# get the index of records with inactive services and NA UPT and VRM.
index_to_drop = initial_merge[initial_merge['UPT'].isna() & initial_merge['VRM'].isna() & (initial_merge['Mode/Type of Service Status'] == 'Inactive')].index

# review the records before you drop them to make sure they are blank
initial_merge.loc[index_to_drop]

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Avg Fares Per Trip FY,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
2,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",CR,PT,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,1.017725,127,8.1,2.6,218492,3517,10,44,0.109,641885
4,8.0,Tri-County Metropolitan Transportation Distric...,Inactive,"Portland, OR--WA",SR,DO,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,0.000000,127,8.1,2.6,218492,3517,10,44,0.109,641885
9,40.0,Central Puget Sound Regional Transit Authority,Inactive,"Seattle--Tacoma, WA",LR,PT,2002,NaN,NaN,Independent Public Agency or Authority of Tran...,...,1.533999,95,8.3,4.9,403896,4783,19,38,0.214,734731
12,58.0,City of Portland,Inactive,"Portland, OR--WA",SR,DO,2002,NaN,NaN,"City, County or Local Government Unit or Depar...",...,0.000000,127,8.1,2.6,218492,3517,10,44,0.109,641885
14,58.0,City of Portland,Inactive,"Portland, OR--WA",TR,DO,2002,NaN,NaN,"City, County or Local Government Unit or Depar...",...,NaN,127,8.1,2.6,218492,3517,10,44,0.109,641885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3251,90033.0,City of Tucson,Inactive,"Tucson, AZ",SR,DO,2024,NaN,NaN,"City, County or Local Government Unit or Depar...",...,0.547189,774,5.6,0.9,80693,1324,5,32,0.027,537263
3253,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2024,NaN,NaN,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
3255,90151.0,Southern California Regional Rail Authority,Inactive,"Los Angeles--Long Beach--Anaheim, CA",CR,DO,2024,NaN,NaN,Independent Public Agency or Authority of Tran...,...,4.839540,195,7.7,2.3,424333,4140,10,52,0.085,3880512
3259,90182.0,Altamont Corridor Express,Inactive,"Stockton, CA",CR,DO,2024,NaN,NaN,Independent Public Agency or Authority of Tran...,...,NaN,1169,4.6,1.3,40460,722,7,17,0.014,312413


In [652]:
# Drop all the inactive service records with NA values
initial_merge = initial_merge.drop(index_to_drop)
initial_merge.shape

(2718, 35)

In [653]:
# View the missing values remaining
na_index = initial_merge[initial_merge['UPT'].isna() & initial_merge['VRM'].isna()].index

initial_merge[['Agency','Mode/Type of Service Status','Mode', 'Year','UPT','VRM']].loc[na_index]

,Agency,Mode/Type of Service Status,Mode,Year,UPT,VRM
1,King County,Active,SR,2002,NaN,NaN
5,Tri-County Metropolitan Transportation Distric...,Active,YR,2002,NaN,NaN
8,Central Puget Sound Regional Transit Authority,Active,LR,2002,NaN,NaN
10,Central Puget Sound Regional Transit Authority,Active,SR,2002,NaN,NaN
11,Alaska Railroad Corporation,Active,AR,2002,NaN,NaN
...,...,...,...,...,...,...
2951,City and County of Honolulu,Active,HR,2022,NaN,NaN
2952,City and County of Honolulu,Active,HR,2022,NaN,NaN
2979,Las Vegas Monorail Company,Active,MG,2022,NaN,NaN
3121,Las Vegas Monorail Company,Active,MG,2023,NaN,NaN


In [654]:
# Get only the remaining NA values
na_values = initial_merge.loc[na_index]

# Find the latest year where UPT and VRM wasn't reported 
agg_function = {'Year':['max']}
unique_pairs = na_values.groupby(['Agency','Mode']).agg(agg_function).reset_index()
pd.set_option('display.max_rows', 62)
display(unique_pairs)

,Agency,Mode,Year
,,,max
0,Alaska Railroad Corporation,AR,2005
1,Altamont Corridor Express,CR,2005
2,Capital Metropolitan Transportation Authority,YR,2011
3,Central Florida Commuter Rail,CR,2013
4,Central Oklahoma Transportation and Parking Au...,SR,2017
5,Central Puget Sound Regional Transit Authority,LR,2002
6,Central Puget Sound Regional Transit Authority,SR,2011
7,City and County of Honolulu,HR,2022
8,City and County of San Francisco,SR,2011


Because we don't have any opening year data, for the 58 services with missing values we'll look up their opening date to see if the latest year missing comes before the opening data or when the service first started reporting. If so, these rows will be dropped because the service hadn't been implemented yet, or reporting hadn't started. 

In [656]:
agencies_with_null = [
    'Alaska Railroad Corporation',
    'Capital Metropolitan Transportation Authority',
    'Central Florida Commuter Rail',
    'Central Oklahoma Transportation and Parking Authority',
    'Central Puget Sound Regional Transit Authority',
    'City and County of Honolulu',
    'City and County of San Francisco',
    'City of Charlotte North Carolina',
    'City of Cincinnati',
    'City of El Paso',
    'City of Galveston ',
    'City of Kenosha',
    'City of Memphis',
    'City of Milwaukee',
    'City of Portland',
    'City of Tucson',
    'Dallas Area Rapid Transit',
    'Denton County Transportation Authority',
    'Denver Regional Transportation District',
    'District Department of Transportation',
    'Hillsborough Area Regional Transit Authority',
    'Kansas City, City of Missouri',
    'King County',
    'Loop Trolley Transportation Development District',
    'M-1 Rail',
    'Massachusetts Bay Transportation Authority',
    'McKinney Avenue Transit Authority',
    'Metro Transit ',
    'Metropolitan Atlanta Rapid Transit Authority',
    'Metropolitan Transit Authority of Harris County, Texas ',
    'New Jersey Transit Corporation',
    'New Orleans Regional Transit Authority',
    'North County Transit District',
    'Northeast Illinois Regional Commuter Railroad Corporation',
    'Northern New England Passenger Rail Authority',
    'Rio Metro Regional Transit District',
    'Rock Region Metropolitan Transit Authority',
    'San Diego Metropolitan Transit System',
    'San Francisco Bay Area Rapid Transit District',
    'Sonoma-Marin Area Rail Transit District',
    'Southeastern Pennsylvania Transportation Authority',
    'Southern California Regional Rail Authority',
    'Transportation District Commission of Hampton Roads',
    'Tri-County Metropolitan Transportation District of Oregon',
    'Utah Transit Authority',
    'Valley Metro Rail, Inc.',
    'West Virginia University'
]

# Drop the records for years before a service open/started reporting
for agency in agencies_with_null:
    index_to_drop = initial_merge[(initial_merge['Agency'] == agency) & initial_merge['UPT'].isna() & initial_merge['VRM'].isna()].index
    initial_merge = initial_merge.drop(index_to_drop)


In [657]:
# View the missing values remaining
na_index = initial_merge[initial_merge['UPT'].isna() & initial_merge['VRM'].isna()].index
initial_merge[['Agency','Mode/Type of Service Status','Mode', 'Year','UPT','VRM']].loc[na_index].sort_values('Agency')

,Agency,Mode/Type of Service Status,Mode,Year,UPT,VRM
136,Altamont Corridor Express,Active,CR,2002,NaN,NaN
562,Altamont Corridor Express,Active,CR,2005,NaN,NaN
2356,Fort Worth Transportation Authority,Active,CR,2018,NaN,NaN
2214,Fort Worth Transportation Authority,Active,CR,2017,NaN,NaN
2072,Fort Worth Transportation Authority,Active,CR,2016,NaN,NaN
1930,Fort Worth Transportation Authority,Active,CR,2015,NaN,NaN
1788,Fort Worth Transportation Authority,Active,CR,2014,NaN,NaN
1504,Fort Worth Transportation Authority,Active,CR,2012,NaN,NaN
1362,Fort Worth Transportation Authority,Active,CR,2011,NaN,NaN
1646,Fort Worth Transportation Authority,Active,CR,2013,NaN,NaN


In [658]:
# create a copy of the dataframe
df_imputed = initial_merge.copy()
df_imputed.shape

(2052, 35)

For our remaining missing values, we're going to impute the median value between the year before the gaps in reporting and the first year reporting resumed. By doing this, we assume that ridership for the missing year(s) will be a value between those two. However, we will be dropping all the records for the Las Vegas Monorail Company because the large amount of missing values makes it hard to impute the data. Additionally, for the Virginia Railway Express, the missing value for the year 2007 is caused by the rail company changing to directly operated instead of purchased transportation type of service, so the value for the year 2007 is included in our data.

In [660]:
# Drop all records for the Las Vegas Monorail Company
index_to_drop = df_imputed[df_imputed['Agency']=='Las Vegas Monorail Company'].index
df_imputed = df_imputed.drop(index_to_drop)
df_imputed.shape

(2024, 35)

In [661]:
# Drop the row with null values for the Virginia Railway Express
index_to_drop = df_imputed[(df_imputed['Agency']=='Virginia Railway Express') & (df_imputed['Year']==2007) & (df_imputed['TOS']=='PT')].index
df_imputed = df_imputed.drop(index_to_drop)
df_imputed.shape

(2023, 35)

In [662]:
# Drop the row for 2002 for the Altamont Corridor Express because there isn't a value before to use to impute
index_to_drop = df_imputed[(df_imputed['Agency']=='Altamont Corridor Express') & (df_imputed['Year']==2002) & df_imputed['UPT'].isna()].index
df_imputed = df_imputed.drop(index_to_drop)

# Impute values for the Altamont Corridor Express
ace_values = df_imputed[(df_imputed['Agency']=='Altamont Corridor Express') & (df_imputed['Year']>=2004) & (df_imputed['Year']<=2006)]
UPT_ace_mean = round(ace_values['UPT'].median())
VRM_ace_mean = round(ace_values['VRM'].median())
ace = df_imputed[df_imputed['Agency']=='Altamont Corridor Express'].copy()
ace['UPT'] = ace['UPT'].fillna(UPT_ace_mean)
ace['VRM'] = ace['VRM'].fillna(VRM_ace_mean)
df_imputed[df_imputed['Agency']=='Altamont Corridor Express'] = ace

In [663]:
# Impute values for the Fort Worth Transportation Authority
fwta_values = df_imputed[(df_imputed['Agency']=='Fort Worth Transportation Authority') & (df_imputed['Year']>=2010) & (df_imputed['Year']<=2019)]
UPT_fwta_mean = round(fwta_values['UPT'].median())
VRM_fwta_mean = round(fwta_values['VRM'].median())
fwta = df_imputed[df_imputed['Agency']=='Fort Worth Transportation Authority'].copy()
fwta['UPT'] = fwta['UPT'].fillna(UPT_fwta_mean)
fwta['VRM'] = fwta['VRM'].fillna(VRM_fwta_mean)
df_imputed[df_imputed['Agency']=='Fort Worth Transportation Authority'] = fwta

In [664]:
# View the remaining null values
df_imputed.isna().sum()

NTD ID                         0
Agency                         0
Mode/Type of Service Status    0
UZA Name                       0
Mode                           0
TOS                            0
Year                           0
UPT                            0
VRM                            2
Organization Type              0
city                           0
state                          0
UZA SQ Miles                   0
UZA Population                 0
Service Area Population        6
Service Area SQ Miles          6
Last Closed Report Year        0
Last Closed FY End Month       0
Last Closed FY End Year        0
Passenger Miles FY             4
Unlinked Passenger Trips FY    4
Avg Trip Length FY             4
Fares FY                       4
Operating Expenses FY          3
Avg Cost Per Trip FY           4
Avg Fares Per Trip FY          4
rank                           0
score                          0
tci                            0
jobs                           0
trips/week

In [665]:
# View the remaining missing VRM records
index_to_drop = df_imputed[df_imputed['VRM'].isna()].index
display(df_imputed.loc[index_to_drop])

# Because there are only 2 records and it'll be difficult to impute due to a lack of data, we'll drop these records
df_imputed = df_imputed.drop(index_to_drop)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Avg Fares Per Trip FY,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
45,30057.0,Pennsylvania Department of Transportation,Active,"Philadelphia, PA--NJ--DE--MD",CR,PT,2002,0.0,NaN,State Government Unit or Department of Transpo...,...,40.817415,400,6.8,3.3,81287,1328,15,29,0.061,50055
1435,58.0,City of Portland,Active,"Portland, OR--WA",TR,PT,2012,0.0,NaN,"City, County or Local Government Unit or Depar...",...,0.580657,127,8.1,2.6,218492,3517,10,44,0.109,641885


In [666]:
# View the remaining rows with null values
missing_rows = df_imputed[df_imputed.isnull().any(axis=1)].index
display(df_imputed.loc[missing_rows])

# Since all these services are inactive, we can assume these null values are a result of that. We can drop these records
df_imputed = df_imputed.drop(missing_rows)

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Avg Fares Per Trip FY,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
129,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2002,24842226.0,6982293.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
135,90182.0,Altamont Corridor Express,Inactive,"Stockton, CA",CR,DO,2002,295810.0,35294.0,Independent Public Agency or Authority of Tran...,...,NaN,1169,4.6,1.3,40460,722,7,17,0.014,312413
271,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2003,25379173.0,6852009.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
277,90182.0,Altamont Corridor Express,Inactive,"Stockton, CA",CR,DO,2003,0.0,0.0,Independent Public Agency or Authority of Tran...,...,NaN,1169,4.6,1.3,40460,722,7,17,0.014,312413
413,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2004,28772441.0,0.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
555,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2005,32132530.0,7631243.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
697,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2006,34380375.0,8090731.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
756,30073.0,Virginia Railway Express,Inactive,"Washington--Arlington, DC--VA--MD",CR,DO,2007,3490998.0,1784231.0,Independent Public Agency or Authority of Tran...,...,NaN,949,5.2,0.7,19953,518,5,10,0.031,42663
839,90054.0,"San Diego Trolley, Inc.",Inactive,"San Diego, CA",LR,DO,2007,36835827.0,7923521.0,?,...,0.825714,625,6.1,1.2,109389,1941,5,20,0.037,1384593
1576,58.0,City of Portland,Inactive,"Portland, OR--WA",TR,DO,2013,808464.0,15783.0,"City, County or Local Government Unit or Depar...",...,NaN,127,8.1,2.6,218492,3517,10,44,0.109,641885


In [667]:
# Check that all null values are gone
df_imputed.isna().sum().sum()

0

In [668]:
# Remove trailing spaces
df_imputed['Agency'] = df_imputed['Agency'].str.strip()

### Drop uncessary columns

In [670]:
df_imputed.head()

,NTD ID,Agency,Mode/Type of Service Status,UZA Name,Mode,TOS,Year,UPT,VRM,Organization Type,...,Avg Fares Per Trip FY,rank,score,tci,jobs,trips/week,routes,transit shed (mi2),%transit,population
0,1.0,King County,Inactive,"Seattle--Tacoma, WA",LR,DO,2002,357935.0,39630.0,"City, County or Local Government Unit or Depar...",...,0.553756,95,8.3,4.9,403896,4783,19,38,0.214,734731
3,8.0,Tri-County Metropolitan Transportation Distric...,Active,"Portland, OR--WA",LR,DO,2002,25665600.0,3198863.0,Independent Public Agency or Authority of Tran...,...,0.898112,127,8.1,2.6,218492,3517,10,44,0.109,641885
6,23.0,City of Seattle,Active,"Seattle--Tacoma, WA",MG,PT,2002,2048787.0,220116.0,"City, County or Local Government Unit or Depar...",...,2.910862,95,8.3,4.9,403896,4783,19,38,0.214,734731
7,40.0,Central Puget Sound Regional Transit Authority,Active,"Seattle--Tacoma, WA",CR,PT,2002,672495.0,286613.0,Independent Public Agency or Authority of Tran...,...,2.850825,95,8.3,4.9,403896,4783,19,38,0.214,734731
16,10003.0,Massachusetts Bay Transportation Authority,Inactive,"Boston, MA--NH",CR,DO,2002,40569600.0,22747143.0,Independent Public Agency or Authority of Tran...,...,3.169348,13,9.1,7.1,732061,5983,14,68,0.306,665942


In [671]:
df_imputed.columns

Index(['NTD ID', 'Agency', 'Mode/Type of Service Status', 'UZA Name', 'Mode',
       'TOS', 'Year', 'UPT', 'VRM', 'Organization Type', 'city', 'state',
       'UZA SQ Miles', 'UZA Population', 'Service Area Population',
       'Service Area SQ Miles', 'Last Closed Report Year',
       'Last Closed FY End Month', 'Last Closed FY End Year',
       'Passenger Miles FY', 'Unlinked Passenger Trips FY',
       'Avg Trip Length FY', 'Fares FY', 'Operating Expenses FY',
       'Avg Cost Per Trip FY', 'Avg Fares Per Trip FY', 'rank', 'score', 'tci',
       'jobs', 'trips/week', 'routes', 'transit shed (mi2)', '%transit',
       'population'],
      dtype='object')

In [672]:
columns_to_delete = {"NTD ID",
                     "Last Closed Report Year", 
                     "Last Closed FY End Month", 
                     "Last Closed FY End Year", 
                     "Passenger Miles FY", 
                     "Unlinked Passenger Trips FY", 
                     "Mode/Type of Service Status",
                     "TOS",
                     "UZA Population",
                    "Service Area Population",
                    "Avg Trip Length FY",
                    "Fares FY",
                    "Operating Expenses FY",
                    "rank",
                    "score",
                    "transit shed (mi2)"}
df_imputed = df_imputed.drop(columns=columns_to_delete)
df_imputed.columns

Index(['Agency', 'UZA Name', 'Mode', 'Year', 'UPT', 'VRM', 'Organization Type',
       'city', 'state', 'UZA SQ Miles', 'Service Area SQ Miles',
       'Avg Cost Per Trip FY', 'Avg Fares Per Trip FY', 'tci', 'jobs',
       'trips/week', 'routes', '%transit', 'population'],
      dtype='object')

### Fix Data Types

In [674]:
df_imputed.dtypes

Agency                    object
UZA Name                  object
Mode                      object
Year                      object
UPT                      float64
VRM                      float64
Organization Type         object
city                      object
state                     object
UZA SQ Miles             float64
Service Area SQ Miles    float64
Avg Cost Per Trip FY     float64
Avg Fares Per Trip FY    float64
tci                      float64
jobs                       int64
trips/week                 int64
routes                     int64
%transit                 float64
population                 int64
dtype: object

In [675]:
# Most of the data types look good, but we'll change some columns to integers instead of floats
df_imputed['Year'] = df_imputed['Year'].astype('int64')
df_imputed['UPT'] = df_imputed['UPT'].astype('int64')
df_imputed['VRM'] = df_imputed['VRM'].astype('int64')

In [676]:
final_df = df_imputed.reset_index(drop=True)

In [677]:
final_df.to_csv('alltransit-railridership_cleaned.csv', header=True, index=False, encoding="utf-8")